# Figure 5

This notebook exposes the code to reproduce the figure 5 of the paper, that represent systematic evaluation of all the models considered in the paper. It may take a while (usually a dozen of hours on a laptop). Indeed, it trains approximately 4000 neural networks.

However, the data to create this figure have been added in the github repository. If you want to simply reproduce the figure, you can check the next notebook [2_Figure5_part2](2_Figure5_part2.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.layers import add as k_add
from tensorflow.keras.layers import multiply as k_multiply
from tensorflow.keras.layers import concatenate as k_concatenate

import scipy
from tqdm.notebook import tqdm
import json

from sklearn.metrics import mean_squared_error
np.random.seed(42)

In [2]:
# System definition
def S(x, tau):
    res = np.exp(-x**2) - 0.1*tau*(np.abs(x)-4.)  
    return res
# data generating process
# Generate Train set
def generate_dataset(p=0.01, N_sample=1000):
    X = np.random.uniform(-4,4,N_sample)
    TAU = np.random.binomial(1, p, size=N_sample)
    XTAU = np.c_[X,TAU]
    Y = S(X,TAU)
    return XTAU, Y, X, TAU

In [3]:
def build_model(nE = 1, nD = 1, ls = 20, enc = "FC", nltype="relu"):
    """
    :param nE: number of layers for encoder E
    :param nD: number of layers for decoder D
    :param ls: size of each layer
    :param enc: one of "FC" / "LEAP" the model considered
    """
    x = Input(shape=(1,))
    tau = Input(shape=(1,))
    
    # build the input of the model
    if enc == "FC" or enc == "ResNet" :
        x_ = k_concatenate([x, tau])  # in the fully connected, the neural network uses a concantenation of x and tau
    elif enc == "LEAP":
        x_ = x
    else:
        raise RuntimeError("Unknown encoding \"{}\"".format(enc))
    tmp = x_

    # enoder (denoted by E)
    for _ in range(nE):
        tmp = Dense(ls)(tmp)
        tmp = Activation(nltype)(tmp)
    E_out = tmp

    # leap part (L_tau)
    if enc == "LEAP":
        tmp = Dense(1)(E_out) # e
        tmp = k_multiply([tau, tmp])  # element wise multiplication
        tmp = Dense(ls, use_bias=False)(tmp) # d
        d_in = k_add([E_out, tmp])
    elif enc == "FC":
        d_in = E_out
    elif enc == "ResNet":
        tmp = Dense(1)(E_out)
        tmp = Activation(nltype)(tmp)
        tmp = Dense(ls)(tmp)
        d_in = k_add([E_out, tmp])
    else:
        raise RuntimeError("Unknown encoding \"{}\"".format(enc))
    
    # decoder (denoted by D)
    tmp = d_in
    for _ in range(nD):
        tmp = Dense(ls)(tmp)
        tmp = Activation(nltype)(tmp)
    D_out = tmp
    
    # last layer
    res_model = Dense(1)(tmp)


    res = Model(inputs=[x, tau], outputs=[res_model])
    return res

def reset_weights(model):
    session = keras.backend.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

In [4]:
# optimizer parameter
lr = 0.001
max_iter = 2000
batch_size=32

# models parameters
nE = 5
nD = 1
ls = 10

# Data parameters
N_sample = 128
p_test = 0.5
N_per_p = 100

## test set
X_TAU_test, Y_test, X_test, TAU_test = generate_dataset(p=p_test, N_sample=N_sample)

In [ ]:
scores = {}
scores_minus1 = {}
scores_plus1 = {}
scores_minus1_train = {}
scores_plus1_train = {}
p_trains = []

scores["leap"] = {}
scores["fc"] = {}
scores["os"] = {}
scores["rn"] = {}
scores_minus1["leap"] = {}
scores_minus1["fc"] = {}
scores_minus1["os"] = {}
scores_minus1["rn"] = {}
scores_plus1["leap"] = {}
scores_plus1["fc"] = {}
scores_plus1["os"] = {}
scores_plus1["rn"] = {}
scores_minus1_train["leap"] = {}
scores_minus1_train["fc"] = {}
scores_minus1_train["os"] = {}
scores_minus1_train["rn"] = {}
scores_plus1_train["leap"] = {}
scores_plus1_train["fc"] = {}
scores_plus1_train["os"] = {}
scores_plus1_train["rn"] = {}


for sample in tqdm(range(N_per_p)):    
    for p_gen in tqdm(range(0,10)):
        p_train = np.exp(-p_gen*0.5)/(1+np.exp(-p_gen*0.5))
        
        if not p_train in scores["leap"]:
            scores["leap"][p_train] = []
            scores["fc"][p_train] = []
            scores["os"][p_train] = []
            scores["rn"][p_train] = []
            
            scores_minus1["leap"][p_train] = []
            scores_plus1["leap"][p_train] = []
            scores_minus1["fc"][p_train] = []
            scores_plus1["fc"][p_train] = []
            scores_minus1["os"][p_train] = []
            scores_plus1["os"][p_train] = []
            scores_minus1["rn"][p_train] = []
            scores_plus1["rn"][p_train] = []
            
            scores_minus1_train["leap"][p_train] = []
            scores_plus1_train["leap"][p_train] = []
            scores_minus1_train["fc"][p_train] = []
            scores_plus1_train["fc"][p_train] = []
            scores_minus1_train["os"][p_train] = []
            scores_plus1_train["os"][p_train] = []
            scores_minus1_train["rn"][p_train] = []
            scores_plus1_train["rn"][p_train] = []
            
        # generate a new training dataset
        X_TAU_train, Y_train, X_train, TAU_train = generate_dataset(p=p_train, N_sample=N_sample)
        
        # build models
        adam_leap = tf.optimizers.Adam(lr=lr)
        leap_model = build_model(nE=nE, nD=nD, ls=ls, enc="LEAP")
        leap_model.compile(optimizer=adam_leap, loss='mse')
        leap_model.fit(x=[X_train, TAU_train], y=[Y_train], epochs=max_iter, verbose=False, batch_size=batch_size)
        Y_pred_leap = leap_model.predict([X_test, TAU_test])
        Y_pred_leap_train = leap_model.predict([X_train, TAU_train])
        tf.keras.backend.clear_session()
        
        adam_fc = tf.optimizers.Adam(lr=lr)
        fc_model = build_model(nE=nE, nD=nD, ls=ls, enc="FC")
        fc_model.compile(optimizer=adam_fc, loss='mse')
        fc_model.fit(x=[X_train, TAU_train], y=[Y_train], epochs=max_iter, verbose=False, batch_size=batch_size)
        Y_pred_fc = fc_model.predict([X_test, TAU_test])
        Y_pred_fc_train = fc_model.predict([X_train, TAU_train])
        tf.keras.backend.clear_session()

        # ResNet fully connected
        adam_rn = tf.optimizers.Adam(lr=lr)
        rn_model = build_model(nE=nE, nD=nD, ls=ls, enc="ResNet")
        rn_model.compile(optimizer=adam_rn, loss='mse')
        rn_model.fit(x=[X_train, TAU_train], y=[Y_train], epochs=max_iter, verbose=False, batch_size=batch_size)
        Y_pred_rn = rn_model.predict([X_test, TAU_test])
        Y_pred_rn_train = rn_model.predict([X_train, TAU_train])
        tf.keras.backend.clear_session()

        # over sampling the cases the less freqent
        adam_os = tf.optimizers.Adam(lr=lr)
        os_model = build_model(nE=nE, nD=nD, ls=ls, enc="ResNet")
        os_model.compile(optimizer=adam_os, loss='mse')
        weights = np.ones(TAU_train.shape, dtype=np.float32)
        weights[TAU_train == 0] = np.float32(1. / (1.-p_train))
        weights[TAU_train == 1] = np.float32(1. / (p_train))
        weights = weights.flatten()
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            # buggy tensorflow 2.1 warning
            # https://github.com/tensorflow/tensorflow/issues/37500
            os_model.fit(x=[X_train, TAU_train],
                         y=[Y_train],
                         epochs=max_iter,
                         verbose=False,
                         batch_size=batch_size,
                         sample_weight=weights)
        Y_pred_os = os_model.predict([X_test, TAU_test])
        Y_pred_os_train = os_model.predict([X_train, TAU_train])
        tf.keras.backend.clear_session()

        scores["leap"][p_train].append(mean_squared_error(Y_test, Y_pred_leap))
        scores_minus1["leap"][p_train].append(mean_squared_error(Y_test[TAU_test==0], Y_pred_leap[TAU_test==0]))
        scores_plus1["leap"][p_train].append(mean_squared_error(Y_test[TAU_test==1], Y_pred_leap[TAU_test==1]))
 
        scores["fc"][p_train].append(mean_squared_error(Y_test, Y_pred_fc))
        scores_minus1["fc"][p_train].append(mean_squared_error(Y_test[TAU_test==0], Y_pred_fc[TAU_test==0]))
        scores_plus1["fc"][p_train].append(mean_squared_error(Y_test[TAU_test==1], Y_pred_fc[TAU_test==1]))
 
        scores["os"][p_train].append(mean_squared_error(Y_test, Y_pred_os))
        scores_minus1["os"][p_train].append(mean_squared_error(Y_test[TAU_test==0], Y_pred_os[TAU_test==0]))
        scores_plus1["os"][p_train].append(mean_squared_error(Y_test[TAU_test==1], Y_pred_os[TAU_test==1]))
 
        scores["rn"][p_train].append(mean_squared_error(Y_test, Y_pred_rn))
        scores_minus1["rn"][p_train].append(mean_squared_error(Y_test[TAU_test==0], Y_pred_rn[TAU_test==0]))
        scores_plus1["rn"][p_train].append(mean_squared_error(Y_test[TAU_test==1], Y_pred_rn[TAU_test==1]))
        
        scores_minus1_train["fc"][p_train].append(mean_squared_error(Y_train[TAU_train==0],
                                                                     Y_pred_fc_train[TAU_train==0]))
        scores_minus1_train["os"][p_train].append(mean_squared_error(Y_train[TAU_train==0],
                                                                       Y_pred_os_train[TAU_train==0]))
        scores_minus1_train["leap"][p_train].append(mean_squared_error(Y_train[TAU_train==0],
                                                                       Y_pred_leap_train[TAU_train==0]))
        scores_minus1_train["rn"][p_train].append(mean_squared_error(Y_train[TAU_train==0],
                                                                       Y_pred_rn_train[TAU_train==0]))
        
        if np.sum(TAU_train==1) >= 1:
            scores_plus1_train["fc"][p_train].append(mean_squared_error(Y_train[TAU_train==1],
                                                                        Y_pred_fc_train[TAU_train==1]))
            scores_plus1_train["leap"][p_train].append(mean_squared_error(Y_train[TAU_train==1],
                                                                          Y_pred_leap_train[TAU_train==1]))
            scores_plus1_train["os"][p_train].append(mean_squared_error(Y_train[TAU_train==1],
                                                                          Y_pred_os_train[TAU_train==1]))
            scores_plus1_train["rn"][p_train].append(mean_squared_error(Y_train[TAU_train==1],
                                                                          Y_pred_rn_train[TAU_train==1]))
        else:
            scores_plus1_train["fc"][p_train].append(-1)
            scores_plus1_train["leap"][p_train].append(-1)
            scores_plus1_train["os"][p_train].append(-1)
            scores_plus1_train["rn"][p_train].append(-1)
            
        with open("scores_5.json", "w") as f:
            json.dump(scores, f, indent=4, sort_keys=True)
        with open("scores_minus1_5.json", "w") as f:
            json.dump(scores_minus1, f, indent=4, sort_keys=True)
        with open("scores_plus1_5.json", "w") as f:
            json.dump(scores_plus1, f, indent=4, sort_keys=True)
        with open("scores_minus1_train_5.json", "w") as f:
            json.dump(scores_minus1_train, f, indent=4, sort_keys=True)
        with open("scores_plus1_train_5.json", "w") as f:
            json.dump(scores_plus1_train, f, indent=4, sort_keys=True)


W0501 12:57:13.892402 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 12:57:59.892874 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 12:58:46.380086 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 12:59:35.241511 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:00:23.274261 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:01:02.199793 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:01:37.685538 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:02:12.385987 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:02:47.5

W0501 13:03:59.902838 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:04:37.526308 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:05:13.028481 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:05:48.236486 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:06:25.200098 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:07:00.680436 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:07:40.356492 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:08:17.703217 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:08:53.0

W0501 13:10:03.981939 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:10:39.039292 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:11:14.642339 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:11:49.750705 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:12:24.999995 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:13:00.721074 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:13:37.381697 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:14:12.869955 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:14:47.7

W0501 13:15:58.151337 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:16:33.679058 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:17:08.724632 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:17:44.562761 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:18:19.615713 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:18:55.111505 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:19:30.679598 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:20:06.033518 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:20:41.3

W0501 13:21:52.816763 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:22:27.990169 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:23:03.388593 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:23:38.687732 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:24:13.792080 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:24:49.185105 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:25:24.488577 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:25:59.914169 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 13:26:35.0

...

W0501 17:30:58.127084 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:31:33.984404 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:32:09.573510 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:32:45.411012 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:33:21.580988 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:34:02.982131 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:34:46.091794 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:35:26.240813 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:36:07.0

W0501 17:37:26.175442 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:38:07.644887 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:38:50.705376 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:39:35.262961 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:40:17.250654 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:40:59.613821 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:41:41.960529 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:42:21.513566 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:43:02.1

W0501 17:44:23.991098 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:45:03.838110 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:45:43.299757 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:46:22.846711 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:47:01.258024 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:47:41.020000 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:48:24.512051 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:49:04.018300 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:49:46.1

W0501 17:51:05.432405 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:51:49.951593 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:52:33.087643 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:53:15.008911 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:53:56.680647 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:54:38.500530 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:55:15.428843 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:55:58.018665 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:56:40.0

W0501 17:58:08.724287 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:58:51.687179 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 17:59:31.251869 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 18:00:17.131133 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 18:01:00.124685 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 18:01:42.628354 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 18:02:26.583883 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0501 18:03:07.983859 139944315049280 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
